<a href="https://colab.research.google.com/github/rajgupt/dl-notebooks/blob/main/pytorch/5_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install pytorch-lightning

     |████████████████████████████████| 819kB 6.8MB/s 
     |████████████████████████████████| 645kB 14.0MB/s 
     |████████████████████████████████| 235kB 24.3MB/s 
     |████████████████████████████████| 829kB 28.1MB/s 
     |████████████████████████████████| 10.6MB 32.5MB/s 
     |████████████████████████████████| 122kB 47.9MB/s 
     |████████████████████████████████| 1.3MB 35.4MB/s 
     |████████████████████████████████| 143kB 46.7MB/s 
     |████████████████████████████████| 296kB 36.3MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=e7136f524764f28f13548900f561721ec7b4652a58b921270983a474d1dfe3a8
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.4.1 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uni

In [10]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

In [12]:
x = torch.rand(1,28,28)
x.size()

torch.Size([1, 28, 28])

In [14]:
x.view(x.size(0),-1).size()

torch.Size([1, 784])

In [15]:
flatten = nn.Flatten()

In [16]:
flatten(x).size()

torch.Size([1, 784])

In [25]:
class LitAutoEncoder(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.encoder = nn.Sequential(
        nn.Linear(28*28, 64),
        nn.ReLU(),
        nn.Linear(64,3)
    )
    self.decoder = nn.Sequential(
        nn.Linear(3,64),
        nn.ReLU(),
        nn.Linear(64,28*28)
    )

  def forward(self,x):
    embedding = self.encoder(x)
    return embedding

  def training_step(self, batch, batch_idx):
    x,y = batch
    x = x.view(x.size(0),-1)
    z = self.encoder(x)
    x_hat = self.decoder(z)
    loss = F.mse_loss(x_hat,x)
    self.log('train_loss',loss)
    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(),lr=1e-3)
    return optimizer

  



In [19]:
dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset)


Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# init model
autoencoder = LitAutoEncoder()

# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer()
trainer.fit(autoencoder, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
